In [1]:
import os
import torch
import imageio.v3 as iio
import numpy as np
import matplotlib.pyplot as plt
from main import SegmentationDataSet
from unet import UNet
import torch
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torch.nn as nn
from torch.optim import Adam
from tqdm import tqdm


/ext3/miniconda3/envs/owl_boto/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# !pip install torchmetrics
import torchmetrics

In [3]:
#Give Unet Saved path here
unet_model_saved_path='/scratch/sd5251/DL/Project/Video-Prediction-and-Segmentation/unet_segmentation/unet_10.pt'
#Give validation set saved path here
val_dir='/scratch/sd5251/DL/Project/clevrer1/dataset/val/video_'

In [4]:
val_data_dir = [val_dir+ str(i) for i in range(1000,2000)]
val_dataset = SegmentationDataSet(val_data_dir,None)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=1, shuffle=True)

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [7]:
model = UNet(n_channels=3, n_classes=49, bilinear=False)
model.to(device)

UNet(
  (inc): DoubleConv(
    (double_conv): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
    )
  )
  (down1): Down(
    (maxpool_conv): Sequential(
      (0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (1): DoubleConv(
        (double_conv): Sequential(
          (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
          (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
 

In [8]:
m = torch.load(unet_model_saved_path).state_dict()

In [10]:
model.load_state_dict(m)

<All keys matched successfully>

In [40]:
jaccard = torchmetrics.JaccardIndex(task="multiclass", num_classes=49).to(device)

In [41]:
def evaluate_jaccard_index(loader, model, device):
    num_correct = 0
    num_pixels = 0
    dice_score = 0
    y_preds_concat = None
    y_trues_concat = None

    y_preds_list = []
    y_trues_list = []
  
    with torch.no_grad():
        for x, y in tqdm(loader):
           
            x = x.permute(0,3,1,2).type(torch.cuda.FloatTensor).to(device)
#             print(x.shape)
#             break
            y = y.to(device)
            softmax = nn.Softmax(dim=1)
            preds = torch.argmax(softmax(model(x)),axis=1)

            y_preds_list.append(preds)
            y_trues_list.append(y)
            
          
            num_correct += ((preds == y) & (y != 0)).sum()
            num_pixels += (y !=0).sum()

            dice_score += (2 * (preds * y).sum()) / ((preds + y).sum() + 1e-8)
           

    y_preds_concat = torch.cat(y_preds_list, dim=0)
    y_trues_concat = torch.cat(y_trues_list, dim=0)

    print(len(y_preds_list))
    print(y_preds_concat.shape)

    jac_idx = jaccard(y_trues_concat, y_preds_concat)

    print(f"Jaccard Index {jac_idx}")

    print(f"Got {num_correct}/{num_pixels} with acc {num_correct/num_pixels*100:.2f}")
    print(f"Dice score: {dice_score/len(loader)}")
#     model.train()

In [46]:
val_data_subset = torch.utils.data.Subset(val_dataset, range(1000))
val_data_subset_loader = torch.utils.data.DataLoader(val_data_subset, batch_size=1, shuffle=True)

In [50]:
# val_data_subset = torch.utils.data.Subset(val_dataset, range(1000))
val_dataset_loader = torch.utils.data.DataLoader(val_dataset, batch_size=1, shuffle=True)

In [47]:
len(val_data_subset_loader)

1000

In [48]:
model.eval();

In [51]:
evaluate_jaccard_index(val_dataset_loader, model, device)

100%|██████████| 2000/2000 [00:33<00:00, 59.63it/s]

2000
torch.Size([2000, 160, 240])
Jaccard Index 0.9176664352416992
Got 3689160/3852578 with acc 95.76
Dice score: 30.141815185546875
